# Phase III: First ML Model Proof of Concept w/ Discussion of Ethical Considerations

In [ ]:
import pandas as pd

In [ ]:
flights_df = pd.read_parquet("cleaned_flights.parquet")

In [ ]:
# Extract the hour from the 'DATE' column and create a new column 'DEPARTURE_HOUR'
flights_df['DEPARTURE_HOUR'] = flights_df['DATE'].dt.hour

# Filter the DataFrame to include only delayed flights
delayed_flights = flights_df[flights_df['ARRIVAL_DELAY'] > 0]
delayed_flights.head()

## Numpy ML Model

In [ ]:
import numpy as np

In [ ]:
# Standardize numeric columns including DISTANCE
for col in ['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK', 'DISTANCE']:
    col_mean = delayed_flights[col].mean()
    col_std  = delayed_flights[col].std()
    delayed_flights[col] = (delayed_flights[col] - col_mean) / col_std

In [ ]:
# Extract features: numeric and categorical features
numeric_feats = delayed_flights[['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK', 'DISTANCE']].values
categorical_feats = pd.get_dummies(delayed_flights[['AIRLINE', 'ORIGIN_AIRPORT']]).values
X = np.hstack([numeric_feats, categorical_feats])
y = delayed_flights['ARRIVAL_DELAY'].values

In [ ]:
# Simple train/test split (e.g. 80% train, 20% test)
split_idx = int(0.8 * len(X))
X_train, y_train = X[:split_idx], y[:split_idx]
X_test,  y_test  = X[split_idx:], y[split_idx:]

In [ ]:
# Add a column of 1s for the intercept
ones_train = np.ones((X_train.shape[0], 1))
X_train_design = np.hstack((ones_train, X_train))

ones_test = np.ones((X_test.shape[0], 1))
X_test_design = np.hstack((ones_test, X_test))

In [ ]:
# Normal equation using pseudo-inverse to handle singular matrix: w = (XᵀX)⁺ Xᵀy
w = np.linalg.pinv(X_train_design.T @ X_train_design) @ (X_train_design.T @ y_train)

# Predictions
y_pred = X_test_design @ w

# Evaluate with MSE
mse = np.mean((y_test - y_pred)**2)
print("MSE:", mse)
print("Intercept:", w[0])
print("Coefficients (MONTH, DEPARTURE_HOUR, DAY_OF_WEEK):", w[1:])

# Compute R²
ss_res = np.sum((y_test - y_pred) ** 2)
ss_tot = np.sum((y_test - np.mean(y_test)) ** 2)
r2 = 1 - (ss_res / ss_tot)
print("R²:", r2)

### Find optimal and least optimal flight features

In [ ]:
# Add intercept
ones = np.ones((X.shape[0], 1))
X_design = np.hstack((ones, X))

# Generate predictions
y_pred = X_design @ w

# Find indices for min/max predicted delays
best_idx = np.argmin(y_pred)
worst_idx = np.argmax(y_pred)

# Retrieve flights
best_flight = delayed_flights.iloc[best_idx]
worst_flight = delayed_flights.iloc[worst_idx]

In [ ]:
print("Best flight (lowest predicted delay):")
print(best_flight[['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK']])
print("Predicted delay (minutes):", y_pred[best_idx])

In [ ]:
print("Worst flight (highest predicted delay):")
print(worst_flight[['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK']])
print("Predicted delay (minutes):", y_pred[worst_idx])

## NumPy Quadratic ML Model
*Note*: It was way too hard to use categorical data with NumPy for non-linear models, so we will do that in the future with sklearn

In [ ]:
# Quadratic feature expansion
month_sq = delayed_flights['MONTH'] ** 2
sched_dep_sq = delayed_flights['DEPARTURE_HOUR'] ** 2
dow_sq = delayed_flights['DAY_OF_WEEK'] ** 2
distance_sq = delayed_flights['DISTANCE'] ** 2
interaction_1 = delayed_flights['MONTH'] * delayed_flights['DEPARTURE_HOUR']
interaction_2 = delayed_flights['MONTH'] * delayed_flights['DAY_OF_WEEK']
interaction_3 = delayed_flights['DEPARTURE_HOUR'] * delayed_flights['DAY_OF_WEEK']
interaction_4 = delayed_flights['MONTH'] * delayed_flights['DISTANCE']
interaction_5 = delayed_flights['DEPARTURE_HOUR'] * delayed_flights['DISTANCE']
interaction_6 = delayed_flights['DAY_OF_WEEK'] * delayed_flights['DISTANCE']

# Combine original and new features
X_quad = np.column_stack([
    delayed_flights['MONTH'],
    delayed_flights['DEPARTURE_HOUR'],
    delayed_flights['DAY_OF_WEEK'],
    delayed_flights['DISTANCE'],
    month_sq,
    sched_dep_sq,
    dow_sq,
    distance_sq,
    interaction_1,
    interaction_2,
    interaction_3,
    interaction_4,
    interaction_5,
    interaction_6
])
y_quad = delayed_flights['ARRIVAL_DELAY'].values

# Train/test split
split_idx = int(0.8 * len(X_quad))
X_train_quad, y_train_quad = X_quad[:split_idx], y_quad[:split_idx]
X_test_quad,  y_test_quad  = X_quad[split_idx:], y_quad[split_idx:]

# Add intercept
ones_train_quad = np.ones((X_train_quad.shape[0], 1))
X_train_design_quad = np.hstack((ones_train_quad, X_train_quad))

ones_test_quad = np.ones((X_test_quad.shape[0], 1))
X_test_design_quad = np.hstack((ones_test_quad, X_test_quad))

# Normal equation for quadratic features
w_quad = np.linalg.inv(X_train_design_quad.T @ X_train_design_quad) @ (X_train_design_quad.T @ y_train_quad)

# Predictions
y_pred_quad = X_test_design_quad @ w_quad
mse_quad = np.mean((y_test_quad - y_pred_quad)**2)
print("Quadratic MSE:", mse_quad)